# **API DataJud CNJ**

O código a seguir visa buscar os processo da Vara Cível, dos Feitos da Fazenda e Registros Públicos de Dianópolis, via API do DataJud do CNJ.

O objetivo é utilizar os dados para análise e gerenciamento estratégico da unidade, com vistas a idetifiticar gargalos que possam gerar atrasos no andamento.

A API não apresenta dados dos processos marcados com "segredo de justiça".

Neste momento, os dados do DataJud estão atualizados até 30/10/2024.

Documentação da API (https://datajud-wiki.cnj.jus.br/api-publica/)

# ***1. Bibliotecas***

Importa as bibliotecas necessárias

In [ ]:
import pandas as pd
import requests
import json

# ***2. Dados da API***

URL da API que busca os dados dos processos do TJTO. Para buscar dados de outros tribunais, basta acessar a página do CNJ (https://datajud-wiki.cnj.jus.br/api-publica/endpoints). A chave pública abaixo funciona para qualquer tribunal.

In [ ]:
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjto/_search"
api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==" # Chave pública

# ***3. Busca dados de um processo***

O trecho abaixo busca os dados de um processo específico. Basta informar o número do processo. Para buscar os dados dos processos do TJTO, é necessário informar o número do processo entre aspas (" "). O código abaixo busca todos os dados do processo, inclusive os movimentos. Os dados são retornados em formato JSON. É importante destacar que são apresentados de todos os graus de jurisdição (1º e 2º grau).

In [18]:
numero_processo = "00034953520178272716" # Informar o número do processo - O processo escolhido para teste foi selecionado aleatoriamente.
payload = json.dumps({
  "query": {
    "match": {
      "numeroProcesso": numero_processo,
      #"classe.codigo": 12729  ---12729 (Exec. de Med. Alternativas) Para buscar os dados referente a uma classe judicial específica
    }
  }
})

headers = {
  'Authorization': api_key,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)  # <Response [200]>
dados = response.json() # <class 'dict'>
print(dados)

{'took': 4, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 13.6470585, 'hits': [{'_index': 'api_publica_tjto', '_id': 'TJTO_G2_00034953520178272716', '_score': 13.6470585, '_source': {'numeroProcesso': '00034953520178272716', 'classe': {'codigo': 198, 'nome': 'Apelação Cível'}, 'sistema': {'codigo': -1, 'nome': 'Inválido'}, 'formato': {'codigo': 1, 'nome': 'Eletrônico'}, 'tribunal': 'TJTO', 'dataHoraUltimaAtualizacao': '2024-08-05T14:52:48.732Z', 'grau': 'G2', '@timestamp': '2024-09-20T11:43:48.731328366Z', 'dataAjuizamento': '2024-06-21T16:02:42.000Z', 'movimentos': [{'complementosTabelados': [{'codigo': 2, 'valor': 2, 'nome': 'sorteio', 'descricao': 'tipo_de_distribuicao_redistribuicao'}], 'codigo': 26, 'nome': 'Distribuição', 'dataHora': '2024-06-21T16:02:42.000Z'}, {'complementosTabelados': [{'codigo': 18, 'valor': 40, 'nome': 'outros motivos', 'descricao': 'motivo_da_remessa'}]

# ***4. Dados básicos do processo***

O trecho a seguir apresenta os dados básicos do processo no 1º grau, como número do processo, classe, assunto, órgão julgador, data do ajuízamento e última atualização. Para buscar os dados do 2º grau, bastar alterar o índice dos dados em cada linha para "0" (ex.: numero_processo = dados['hits']['hits']`[0]`['_source']['numeroProcesso']).

In [19]:
numero_processo = dados['hits']['hits'][1]['_source']['numeroProcesso']
classe = dados['hits']['hits'][1]['_source']['classe']['nome']
data_ajuizamento = dados['hits']['hits'][1]['_source']['dataAjuizamento']
ultima_atualizacao = dados['hits']['hits'][1]['_source']['dataHoraUltimaAtualizacao']
codigo_orgao_julgador = dados['hits']['hits'][1]['_source']['orgaoJulgador']['codigo']
orgao_julgador = dados['hits']['hits'][1]['_source']['orgaoJulgador']['nome']
movimentos = dados['hits']['hits'][1]['_source']['movimentos']
assunto = dados['hits']['hits'][1]['_source']['assuntos'][0]['nome']

print(f'Número do processo: {numero_processo}')
print(f'Classe: {classe}')
print(f'Assunto principal: {assunto}')
print(f'Órgão julgador: {codigo_orgao_julgador} - {orgao_julgador}')
print(f'Data de ajuizamento:{data_ajuizamento}')
print(f'Última atualização: {ultima_atualizacao}')

Número do processo: 00034953520178272716
Classe: Procedimento Comum Cível
Assunto principal: Honorários Advocatícios
Órgão julgador: 85355 - Juízo da Vara Cível, dos Feitos da Fazenda e Registros Públicos de Dianópolis
Data de ajuizamento:2017-12-14T14:54:59.000Z
Última atualização: 2024-09-19T15:28:02.153Z


# ***4.1 - Dados do primeiro grau***

Quando o processo foi remetido ao segundo grau para análise de recurso, a API traz todas as informações de ambos os recursos.

Nesse sentido, o código abaixo apresenta os dados referente apenas ao primeiro grau, que até o momento, é o que nos interessa.

In [32]:
for processos_1_grau in dados['hits']['hits'][::-1]:
    if processos_1_grau['_source']['grau'] == 'G1':
      print(f'Processo de 1º grau')
      print(f'Número do processo: {processos_1_grau["_source"]["numeroProcesso"]}')
      print(f'Classe: {processos_1_grau["_source"]["classe"]["nome"]}')

Processo de 1º grau
Número do processo: 00034953520178272716
Classe: Procedimento Comum Cível


# ***5. Movimentos***

Apresenta todos os movimentos do processo acima seleciondo. Os dados não estão apresentados por ordem cronológica (ainda vou melhorar esse trecho do código).

In [ ]:
# Movimentos
print(f'Número do processo: {numero_processo}')
print()
for movimento in movimentos[::-1]:
    codigo = movimento.get('codigo')
    nome = movimento.get('nome')
    data_hora = movimento.get('dataHora')
    print(f"Data/Hora: {data_hora}\t Código: {codigo}\t Nome: {nome}")

Número do processo: 00034953520178272716

Data/Hora: 2024-07-29T23:59:59.000Z	 Código: 12266	 Nome: Confirmada
Data/Hora: 2024-07-19T11:48:41.000Z	 Código: 12265	 Nome: Expedida/certificada
Data/Hora: 2024-07-19T11:48:40.000Z	 Código: 12265	 Nome: Expedida/certificada
Data/Hora: 2024-07-17T14:20:52.000Z	 Código: 132	 Nome: Recebimento
Data/Hora: 2024-07-09T16:03:09.000Z	 Código: 12282	 Nome: Expedida/Certificada
Data/Hora: 2024-06-21T15:00:08.000Z	 Código: 11010	 Nome: Mero expediente
Data/Hora: 2024-06-06T23:59:59.000Z	 Código: 12266	 Nome: Confirmada
Data/Hora: 2024-05-27T13:16:06.000Z	 Código: 12265	 Nome: Expedida/certificada
Data/Hora: 2024-05-27T13:16:04.000Z	 Código: 12265	 Nome: Expedida/certificada
Data/Hora: 2024-05-21T12:56:03.000Z	 Código: 132	 Nome: Recebimento
Data/Hora: 2020-06-15T08:15:58.000Z	 Código: 12282	 Nome: Expedida/Certificada
Data/Hora: 2020-05-12T08:12:28.000Z	 Código: 12282	 Nome: Expedida/Certificada
Data/Hora: 2019-09-27T10:46:07.000Z	 Código: 12282	 Nome:

# ***6. Busca todos os processos da comarca***

O trecho a seguir busca todos os processos da unidade informada (Vara Cível de Dianópolis), ordenados por data do ajuizamento (ordem crescente). Armazena os dados em formato JSON.

Também podemos buscar todos os processos daquela unidade por distrito. Para tanto, basta descomentar a linha e informar o código do município no IBGE.

In [ ]:
orgao_julgador = 85355 #Código da Vara Cível da Comarca de Dianópolis

payload = json.dumps({
  "size": 10000,
  "query": {
    "match": {"orgaoJulgador.codigo": orgao_julgador,
             #"orgaoJulgador.codigoMunicipioIBGE": 3538709 //Com essa linha pode-se buscar os processos de um distrito da comarca
    }
  },
  "sort": [{"dataAjuizamento": {"order": "asc"}}] # ou desc - Ordena os processos por ordem de distribuição
})

headers = {
  'Authorization': api_key,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
dados = json.loads(response.text)

# ***7. Dados do processo***

Após buscar todos os processos da unidade, apresentamos apenas os dados do primeiro processo da lista em formato JSON.

In [ ]:
dados['hits']['hits'][0]

{'_index': 'api_publica_tjto',
 '_id': 'TJTO_G1_50000013019878272716',
 '_score': None,
 '_source': {'numeroProcesso': '50000013019878272716',
  'classe': {'codigo': 1116, 'nome': 'Execução Fiscal'},
  'sistema': {'codigo': -1, 'nome': 'Inválido'},
  'formato': {'codigo': 1, 'nome': 'Eletrônico'},
  'tribunal': 'TJTO',
  'dataHoraUltimaAtualizacao': '2024-09-19T06:28:35.084Z',
  'grau': 'G1',
  '@timestamp': '2024-09-25T02:28:48.068688562Z',
  'dataAjuizamento': '1987-04-29T00:00:00.000Z',
  'movimentos': [{'complementosTabelados': [{'codigo': 2,
      'valor': 85,
      'nome': 'sorteio manual',
      'descricao': 'tipo_de_distribuicao_redistribuicao'}],
    'codigo': 26,
    'nome': 'Distribuição',
    'dataHora': '2013-11-22T15:50:40.000Z'},
   {'complementosTabelados': [{'codigo': 19,
      'valor': 57,
      'nome': 'Petição (outras)',
      'descricao': 'tipo_de_peticao'}],
    'codigo': 118,
    'nome': 'Protocolo de Petição',
    'dataHora': '2013-11-29T11:32:46.000Z'},
   {'co

**8. Lista de processos**

Agora criamos uma lista para armazenar os dados de todos dos processos da unidade. Ao final, armazena os dados em um dataframe.


In [ ]:
# Lista para armazenar os dados dos processos
processos = []

for processo in dados['hits']['hits']:
  numero_processo = processo['_source']['numeroProcesso']
  grau = processo['_source']['grau']
  classe = processo['_source']['classe']['nome']
  assuntos = processo['_source']['assuntos'] # Pode ter mais de um
  data_ajuizamento = processo['_source']['dataAjuizamento']
  ultima_atualizacao = processo['_source']['dataHoraUltimaAtualizacao']
  formato = processo['_source']['formato']['nome']
  codigo = processo['_source']['orgaoJulgador']['codigo']
  orgao_julgador = processo['_source']['orgaoJulgador']['nome']
  municipio = processo['_source']['orgaoJulgador']['codigoMunicipioIBGE']
  sort = processo['sort'][0]
  try:
    movimentos = processo['_source']['movimentos']
  except:
    movimentos = []

  processos.append([numero_processo, classe, data_ajuizamento, ultima_atualizacao, formato, \
                    codigo, orgao_julgador, municipio, grau, assuntos, movimentos, sort])

df = pd.DataFrame(processos, columns=['numero_processo', 'classe', 'data_ajuizamento', 'ultima_atualizacao', \
                      'formato', 'codigo', 'orgao_julgador', 'municipio', 'grau', 'assuntos', 'movimentos', 'sort'])

# Exibindo o DataFrame
display(df)

,numero_processo,classe,data_ajuizamento,ultima_atualizacao,formato,codigo,orgao_julgador,municipio,grau,assuntos,movimentos,sort
0,50000013019878272716,Execução Fiscal,1987-04-29T00:00:00.000Z,2024-09-19T06:28:35.084Z,Eletrônico,85355,"Juízo da Vara Cível, dos Feitos da Fazenda e R...",0,G1,"[{'codigo': 6017, 'nome': 'Dívida Ativa (Execu...","[{'complementosTabelados': [{'codigo': 2, 'val...",546652800000
1,50000017519878272701,Execução Fiscal,1987-08-28T00:00:00.000Z,2024-08-21T10:53:27.029Z,Eletrônico,85355,"Juízo da Vara Cível, dos Feitos da Fazenda e R...",0,G1,"[{'codigo': 7709, 'nome': 'Compensação'}]","[{'complementosTabelados': [{'codigo': 2, 'val...",557107200000
2,50000059619898272716,Procedimento Comum Cível,1989-05-04T00:00:00.000Z,2024-07-23T20:23:12.514Z,Eletrônico,85355,"Juízo da Vara Cível, dos Feitos da Fazenda e R...",0,G1,"[{'codigo': 4703, 'nome': 'Defeito, nulidade o...","[{'complementosTabelados': [{'codigo': 2, 'val...",610243200000
3,50000076619898272716,Cumprimento de sentença,1989-05-04T00:00:00.000Z,2024-09-19T06:15:56.852Z,Eletrônico,85355,"Juízo da Vara Cível, dos Feitos da Fazenda e R...",0,G1,"[{'codigo': 4703, 'nome': 'Defeito, nulidade o...","[{'complementosTabelados': [{'codigo': 2, 'val...",610243200000
4,50000041419898272716,Procedimento Comum Cível,1989-05-04T00:00:00.000Z,2024-07-23T20:26:28.158Z,Eletrônico,85355,"Juízo da Vara Cível, dos Feitos da Fazenda e R...",0,G1,"[{'codigo': 4703, 'nome': 'Defeito, nulidade o...","[{'complementosTabelados': [{'codigo': 2, 'val...",610243200000
...,...,...,...,...,...,...,...,...,...,...,...,...
8739,00021542720248272716,Usucapião,2024-08-28T14:36:12.000Z,2024-09-19T15:37:29.860Z,Eletrônico,85355,"Juízo da Vara Cível, dos Feitos da Fazenda e R...",0,G1,"[{'codigo': 10458, 'nome': 'Usucapião Extraord...","[{'complementosTabelados': [{'codigo': 2, 'val...",1724855772000
8740,00021611920248272716,Procedimento Comum Cível,2024-08-28T20:55:01.000Z,2024-09-19T15:41:03.494Z,Eletrônico,85355,"Juízo da Vara Cível, dos Feitos da Fazenda e R...",0,G1,"[{'codigo': 15048, 'nome': 'Superendividamento'}]","[{'complementosTabelados': [{'codigo': 2, 'val...",1724878501000
8741,00021777020248272716,Procedimento Comum Cível,2024-08-30T09:43:58.000Z,2024-09-19T15:34:59.509Z,Eletrônico,85355,"Juízo da Vara Cível, dos Feitos da Fazenda e R...",0,G1,"[{'codigo': 7781, 'nome': 'Protesto Indevido d...","[{'complementosTabelados': [{'codigo': 2, 'val...",1725011038000
8742,00021802520248272716,Cumprimento de Sentença contra a Fazenda Pública,2024-08-30T15:33:29.000Z,2024-09-19T15:26:43.203Z,Eletrônico,85355,"Juízo da Vara Cível, dos Feitos da Fazenda e R...",0,G1,"[{'codigo': 13537, 'nome': 'Sucumbenciais '}]","[{'complementosTabelados': [{'codigo': 2, 'val...",1725032009000


In [ ]:
def converte_data(data_str):
    return pd.to_datetime(data_str).tz_convert('America/Sao_Paulo')


def gera_lista_assuntos(assuntos_do_df):
    lst_assuntos=[]
    for assunto in assuntos_do_df:
        try:
            lst_assuntos.append(assunto.get('nome'))
        except:
            lst_assuntos.append('')

    return lst_assuntos


def gera_lista_movimentos(movimentos):
    lst_movimentos_final =[]
    for movimento in movimentos:
        codigo = movimento.get('codigo')
        nome = movimento.get('nome')
        data_hora = movimento.get('dataHora')
        if data_hora:
            data_hora = converte_data(data_hora)
        lst_movimentos_final.append([codigo, nome, data_hora])
    return lst_movimentos_final

In [ ]:
df['assuntos'] = df['assuntos'].apply(gera_lista_assuntos)
df['movimentos'] = df['movimentos'].apply(gera_lista_movimentos)
df['data_ajuizamento'] = df['data_ajuizamento'].apply(converte_data)
df['ultima_atualizacao'] = df['ultima_atualizacao'].apply(converte_data)
df['movimentos']= df['movimentos'].apply(lambda x: sorted(x, key=lambda tup: tup[2], reverse=False))
df.sample(5)

,numero_processo,classe,data_ajuizamento,ultima_atualizacao,formato,codigo,orgao_julgador,municipio,grau,assuntos,movimentos,sort
523,50001529220078272716,Usucapião,2007-04-22 21:00:00-03:00,2024-05-05 23:20:19.756000-03:00,Eletrônico,85355,"Juízo da Vara Cível, dos Feitos da Fazenda e R...",0,G1,[Usucapião Ordinária],"[[26, Distribuição, 2014-09-02 10:44:59-03:00]...",1177286400000
8736,00021360620248272716,Procedimento Comum Cível,2024-08-26 14:33:17-03:00,2024-09-19 12:28:12.679000-03:00,Eletrônico,85355,"Juízo da Vara Cível, dos Feitos da Fazenda e R...",0,G1,"[Revisão de Juros Remuneratórios, Capitalizaçã...","[[26, Distribuição, 2024-08-26 14:33:17-03:00]...",1724693597000
3473,00017441320178272716,Carta Precatória Cível,2017-07-12 12:44:20-03:00,2024-05-28 12:24:14.692000-03:00,Eletrônico,85355,"Juízo da Vara Cível, dos Feitos da Fazenda e R...",0,G1,[Dívida Ativa (Execução Fiscal)],"[[26, Distribuição, 2017-07-12 12:44:20-03:00]...",1499874260000
5503,00025894020208272716,Cumprimento de Sentença contra a Fazenda Pública,2020-03-24 14:30:28-03:00,2024-09-19 12:40:09.168000-03:00,Eletrônico,85355,"Juízo da Vara Cível, dos Feitos da Fazenda e R...",0,G1,[Promoção / Ascensão],"[[26, Distribuição, 2020-03-24 14:30:28-03:00]...",1585071028000
2615,00002837420158272716,Procedimento Comum Cível,2015-02-21 15:56:41-02:00,2024-07-23 17:50:40.581000-03:00,Eletrônico,85355,"Juízo da Vara Cível, dos Feitos da Fazenda e R...",0,G1,"[Defeito, nulidade ou anulação]","[[26, Distribuição, 2015-02-21 15:56:41-02:00]...",1424541401000


In [ ]:
# Gravação dos arquivos

df.to_csv('processos.csv', sep=',', header=True, index=False)

with open('resultado.txt', 'w') as file:
    for index, row in df.iterrows():
        file.write(f"Autos n. {row['numero_processo']}, Classe: {row['classe']}\n")
        file.write(f"Ajuizamento: {row['data_ajuizamento']}\n")
        file.write(f"Assuntos: {row['assuntos']}\n\n")
        for movimento in row['movimentos']:
            file.write(f"{str(movimento[2])} | Cód: {str(movimento[0])} | Mov.: {movimento[1]} \n")
        file.write('\n-----------------------------------------------------------------------------\n\n')